In [1]:
!pip install scrapy
import scrapy
from scrapy.crawler import CrawlerProcess
import pandas as pd

class WikiSpider(scrapy.Spider):
    name = "WS"
    
    # Here is where the API call is inserted
    # start_urls = [
        #'https://projecta.com/about/our-team/'
        #]
        
    start_urls = [
        'https://mobile.twitter.com/hashtag/cats'
        ]
        
        

    # Identifying the desired information from the query response and using xpath to extract
    
    def parse(self, response):
        tweets = response.xpath('//table[@class="tweet  "]/@href').getall()    
        logging.info(f'{len(tweets)} tweets found')    
        for tweet_id in tweets:        
            tweet_id = re.findall("\d+", tweet_id)[-1]        
            tweet_url = 'https://twitter.com/anyuser/status/'+str(tweet_id)           
            yield scrapy.Request(tweet_url, callback=self.parse_tweet)
            
        # for item in response.xpath('//lh'):
            # The ns code identifies the type of page the link comes from, and '0' means it is a Wikipedia entry
            # Other codes indicate links from 'Talk' pages, etc.  Only entries are of interest here, so filtering is done
            # if item.xpath('@ns').extract_first() == '0':
                #yield {
                    #'title': item.xpath('@title').extract_first() }
                    
                    
                
                    
        # Getting the information required to continue to the next ten entries
        # next_page = response.xpath('continue/@lhcontinue').extract_first()
        
        # Recursively calling the spider to process the next ten entries, if they exist
        # if next_page is not None:
            #next_page = '{}&lhcontinue={}'.format(self.start_urls[0],next_page)
            #yield scrapy.Request(next_page, callback=self.parse)
            
            
        next_page = response.xpath('//*[@class="w-button-more"]/a/@href').get(default='')
        logging.info('Next page found:')
        if next_page != '':
            next_page = 'https://mobile.twitter.com' + next_page
            yield scrapy.Request(next_page, callback=self.find_tweets)
            
            
            
            
            
            
    
process = CrawlerProcess({
    'FEED_FORMAT': 'json',
    'FEED_URI': 'PythonLinks.json',
    
    
     # Note that because the API queries are being done, the robots.txt file doesn't apply
    'ROBOTSTXT_OBEY': False,
    'USER_AGENT': 'ThinkfulDataScienceBootcampCrawler (thinkful.com)',
    'AUTOTHROTTLE_ENABLED': True,
    'HTTPCACHE_ENABLED': True,
    'LOG_ENABLED': False,
    # Use CLOSESPIDER_PAGECOUNT to limit the scraper to the first 100 links    
    'CLOSESPIDER_PAGECOUNT' : 10
})
                                         

# Starting the crawler with our spider.
process.crawl(WikiSpider)
process.start()
print('First 100 links extracted!')

First 100 links extracted!


In [2]:
Monty=pd.read_json('PythonLinks.json', lines=True)
print(Monty.shape)
print(Monty.tail())

(0, 0)
Empty DataFrame
Columns: []
Index: []
